In [1]:
import random, re
from collections import defaultdict
import itertools
import pycosat
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, BasicAer, IBMQ, execute
from qiskit.tools.visualization import plot_histogram

In [3]:
def tcnfgen(n,k,horn=1):
    cnf = []
    def unique(l,k):
        t = random.randint(1,n)
        while(t in l):
            t = random.randint(1,n)
        return t
    r = (lambda : random.randint(0,1))
    def r_to_sign(x):
        if r() == 1:
            return x
        else:
            return -x
    for i in range(k):
        x = unique([],n)
        y = unique([x],n)
        z = unique([x, y],n)
        if horn:
            cnf.append([x, -y,-z])
        else:
            cnf.append([r_to_sign(x), r_to_sign(y),r_to_sign(z)])
    return cnf

In [4]:
def create_CNF(n, k):
    new_line = []
    line = tcnfgen(n, k)
    for clause in line:
        new_line.append(sorted(clause, key = abs))
    return new_line

In [5]:
def prepare_clause(circuit, quantum_reg, clause):
    for element in clause:
        if element<0:
            circuit.x(quantum_reg[abs(element)])
    return circuit

In [6]:
def or_operator(circuit,quantum_reg, clause, counter):
    order_clause = [abs(x) for x in clause]
    for element in order_clause:
        circuit.x(quantum_reg[element])
    circuit.mcx(order_clause, quantum_reg[counter])
    circuit.x(quantum_reg[counter])
    for element in order_clause:
        circuit.x(quantum_reg[element])
    return circuit

In [7]:
def oracle(circuit, n, k,quantum_reg, CNF):
    counter = n+1 
    for clause in CNF:
        prepare_clause(circuit,quantum_reg, clause)
        or_operator(circuit,quantum_reg, clause, counter)
        counter = counter + 1
        prepare_clause(circuit,quantum_reg, clause)
    circuit.mcx(quantum_reg[n+1:counter], quantum_reg[0])
    for qubit in range(n+1, counter):
        circuit.reset(qubit)
    return circuit

In [8]:
def inversion(circuit,quantum_reg, n):
    for i in range(1,n+1):
        circuit.h(quantum_reg[i])

    for i in range(1,n+1):
        circuit.x(quantum_reg[i])
    
    circuit.mcx(qreg[1:n], quantum_reg[0])

    for i in range(1,n+1):
        circuit.x(quantum_reg[i])

    for i in range(1,n+1):
        circuit.h(quantum_reg[i])

    circuit.x(quantum_reg[0])

In [9]:
n = 3
k = 5
qubit_count = n+1
maxN = qubit_count

new_line = [[-1,-2,-3],[-1,-2,3],[-1,2,-3],[-1,2,3],[1,-2,-3]]

In [10]:
for sol in pycosat.itersolve(new_line):
    print(sol)

[-1, -2, -3]
[-1, -2, 3]
[-1, 2, -3]


In [11]:
"""qreg = QuantumRegister(n + k + 1)  # quantum register with max(n+2*k+2+k,2*n+1) qubits
creg = ClassicalRegister(n + k + 1)  # classical register with max(n+2*k+2+k,2*n+1) bits
mycircuit = QuantumCircuit(qreg, creg)

for i in range(1, n+1):
    mycircuit.h(qreg[i])

oracle(mycircuit,n, k,qreg, new_line)
mycircuit.measure(qreg, creg)

job = execute(mycircuit, Aer.get_backend('qasm_simulator'), shots=1000)
#job = execute(mycircuit, backend= simulator_backend, shots=8192)
counts = job.result().get_counts(mycircuit)

# print the reverse of the outcome
for outcome in counts:
    reverse_outcome = ''
    for i in outcome:
        reverse_outcome = i + reverse_outcome
    print(reverse_outcome, "is observed", counts[outcome], "times")
print("\n")"""

'qreg = QuantumRegister(n + k + 1)  # quantum register with max(n+2*k+2+k,2*n+1) qubits\ncreg = ClassicalRegister(n + k + 1)  # classical register with max(n+2*k+2+k,2*n+1) bits\nmycircuit = QuantumCircuit(qreg, creg)\n\nfor i in range(1, n+1):\n    mycircuit.h(qreg[i])\n\noracle(mycircuit,n, k,qreg, new_line)\nmycircuit.measure(qreg, creg)\n\njob = execute(mycircuit, Aer.get_backend(\'qasm_simulator\'), shots=1000)\n#job = execute(mycircuit, backend= simulator_backend, shots=8192)\ncounts = job.result().get_counts(mycircuit)\n\n# print the reverse of the outcome\nfor outcome in counts:\n    reverse_outcome = \'\'\n    for i in outcome:\n        reverse_outcome = i + reverse_outcome\n    print(reverse_outcome, "is observed", counts[outcome], "times")\nprint("\n")'

In [12]:
qreg = QuantumRegister(n + k + 1) 
creg = ClassicalRegister(n + k + 1)
mycircuit = QuantumCircuit(qreg, creg)

for i in range(1, n+1):
    mycircuit.h(qreg[i])
    
iterations = 3
mycircuit.x(qreg[0])
mycircuit.h(qreg[0])

for i in range(iterations):
    #query
    oracle(mycircuit,n, k,qreg, new_line)
    mycircuit.barrier()
    #inversion
    inversion(mycircuit,qreg, n)
    mycircuit.barrier()


mycircuit.h(qreg[0])
mycircuit.x(qreg[0]) 
mycircuit.measure(qreg, creg)
job = execute(mycircuit, Aer.get_backend('qasm_simulator'), shots=10000)
#job = execute(mycircuit, backend= simulator_backend, shots=8192)
counts = job.result().get_counts(mycircuit)

# print the reverse of the outcome
for outcome in counts:
    reverse_outcome = ''
    for i in outcome:
        reverse_outcome = i + reverse_outcome
    print(reverse_outcome, "is observed", counts[outcome], "times")
print("\n")

011100000 is observed 1233 times
010100000 is observed 1210 times
011000000 is observed 1697 times
010000000 is observed 1776 times
000000000 is observed 785 times
000100000 is observed 1235 times
001100000 is observed 1305 times
001000000 is observed 759 times


